# Experiments
This notebook contains experiments ran on the entire dataset. It contains a log of all the recommenders we discuss during our final presentation. We utilise both the v2 and v3 dataset. The hyperparamater optimisation and other experiments that do not need to be run on the entire dataset.

In [6]:
from src.recommenders import ImprovedRecommender, PopBasedRecommender
import numpy as np
import pickle

In [2]:
def save_pickle(save_path: str, data:object) -> None:
    """Save object to pickle file

    Args:
        save_path (str): filepath to save location
        data (object): object to be saved
    """
    with open(save_path, "wb") as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [3]:
def invert_dict(dicts:list) -> dict:
    """Converts list of dicts to dict of lists

    Args:
        dicts (list): list containing dicts to be inverted

    Returns:
        dict: inverted dict that contains a list of all values in the previous dicts (in order)
    """
    new_dict = dict()
    for i, d in enumerate(dicts):
        for key in d.keys():
            ar = new_dict.get(key, np.zeros(len(dicts)))
            ar[i] = d[key]
            new_dict[key] = ar
    return new_dict

## Parameters
The generation parameters all our recommenders will use.

In [4]:
am_splits = 5           # amount of splits to consider
am_recommended = 20     # amount of items to recommend
k_values = [5, 10, 20]  # k values for metrics to be considered, max should be lesser or equal to am_recommended

# Old data (v2)
The data used for these test is the complete dataset, as provided by group 10.
- Duplicate items are dropped
- Unowned items are dropped
- Users with less than 3 items are dropped

We need at least 3 items for each users to ensure that none of the train/validation/test splits are empty.

In [5]:
data_basepath = "./data/v2/"
quant_output_path = "./evaluation/v2/quantitative/"
qual_output_path = "./evaluation/v2/qualitative/"
items_path = data_basepath + "games.pkl"
reviews_path = data_basepath + "reviews.parquet"
data_path = data_basepath + "interactions_splits_"

## Popularity based recommender
First we create our baseline, this recommender will simply give the first `am_recommended` most frequently occurring items the user doesn't already have in it's training dataset.

In [6]:
evaluations = list()

for split in range(am_splits):
    rec = PopBasedRecommender(train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet", val_path=f"{data_path}{split}_val.parquet")
    rec.generate_recommendations(amount=am_recommended)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
        
evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "PopBased.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.50643163, 0.50444656, 0.50382722, 0.50357313, 0.50558996]),
 'nDCG@5': array([0.18972128, 0.18966486, 0.19010971, 0.18954848, 0.19111418]),
 'recall@5': array([0.15457402, 0.15466747, 0.15479303, 0.1544612 , 0.1565749 ]),
 'ideal_recall@5': array([0.78360788, 0.78360788, 0.78360788, 0.78360788, 0.78360788]),
 'nRecall@5': array([0.20047324, 0.20037425, 0.20040178, 0.19983299, 0.20214679]),
 'HR@10': array([0.63258695, 0.63071304, 0.63049071, 0.63109417, 0.63385739]),
 'nDCG@10': array([0.19870568, 0.19873141, 0.19913717, 0.19852995, 0.20022896]),
 'recall@10': array([0.21966529, 0.21943092, 0.22015371, 0.21925797, 0.22141995]),
 'ideal_recall@10': array([0.91868049, 0.91868049, 0.91868049, 0.91868049, 0.91868049]),
 'nRecall@10': array([0.23557168, 0.23525505, 0.23582393, 0.23489248, 0.23715273]),
 'HR@20': array([0.74734   , 0.74595839, 0.74622836, 0.74662538, 0.74997618]),
 'nDCG@20': array([0.22531723, 0.22525447, 0.22569324, 0.22517075, 0.22728064]),
 'recall@20':

## Base recommender
Runs our recommender at the same settings used in the first presentation. This will be used to compare against our previous results, since our BaseRecommender is no longer compatible with the new data structure.

In [6]:
evaluations = list()
use_data = ["genres", "tags"]
for split in range(am_splits):
    rec = ImprovedRecommender(items_path, train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet",
                              val_path=f"{data_path}{split}_val.parquet", reviews_path=reviews_path, tfidf="smooth", normalize=True, columns=use_data, weighting_scheme=None)
    rec.generate_recommendations(amount=am_recommended, silence=True)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
    rec.qualitative_evaluation(export_path=qual_output_path + f"BaseRecommender_{split}.csv")

evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "BaseRecommender.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.16592028, 0.16760362, 0.16530094, 0.16666667, 0.16842941]),
 'nDCG@5': array([0.04695587, 0.04740548, 0.04717323, 0.04755501, 0.04798619]),
 'recall@5': array([0.04144241, 0.04217006, 0.04131256, 0.04236316, 0.04202838]),
 'ideal_recall@5': array([0.78360788, 0.78360788, 0.78360788, 0.78360788, 0.78360788]),
 'nRecall@5': array([0.05356069, 0.05432693, 0.05337727, 0.05431158, 0.05425255]),
 'HR@10': array([0.26892171, 0.26841353, 0.26866762, 0.26957281, 0.27038272]),
 'nDCG@10': array([0.0554832 , 0.05555438, 0.05568706, 0.0561207 , 0.05607691]),
 'recall@10': array([0.06960315, 0.06949652, 0.06943631, 0.07071003, 0.06936432]),
 'ideal_recall@10': array([0.91868049, 0.91868049, 0.91868049, 0.91868049, 0.91868049]),
 'nRecall@10': array([0.07454448, 0.07438279, 0.07430855, 0.07552353, 0.07428829]),
 'HR@20': array([0.4033508 , 0.40241385, 0.4026203 , 0.40374782, 0.40215976]),
 'nDCG@20': array([0.06998313, 0.06996441, 0.07024008, 0.07081893, 0.07025983]),
 'recall@20':

## Improved recommender
We explore the effect of different "improvements" we are making.

First we increase the data available in our item representations, we utilise the optimal combination we have found for our dataset which were:
- specs
- publisher
- developer
- tags

In [7]:
evaluations = list()
for split in range(am_splits):
    rec = ImprovedRecommender(items_path, train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet",
                              val_path=f"{data_path}{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf="smooth", normalize=True)
    rec.generate_recommendations(amount=am_recommended, silence=True)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
    rec.qualitative_evaluation(export_path=qual_output_path + f"ImprovedRec_none_{split}.csv")    

evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "ImprovedRec_none.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.32188344, 0.32170875, 0.3235509 , 0.32232809, 0.32204224]),
 'nDCG@5': array([0.10384686, 0.10362487, 0.10384973, 0.10368016, 0.10313948]),
 'recall@5': array([0.08664033, 0.08662051, 0.08632493, 0.08666222, 0.08536851]),
 'ideal_recall@5': array([0.78360788, 0.78360788, 0.78360788, 0.78360788, 0.78360788]),
 'nRecall@5': array([0.1119856 , 0.11171246, 0.11162511, 0.11184823, 0.11051162]),
 'HR@10': array([0.45212006, 0.45029379, 0.45299349, 0.45462919, 0.45342226]),
 'nDCG@10': array([0.11576021, 0.11513865, 0.11585569, 0.11598971, 0.11546148]),
 'recall@10': array([0.13676537, 0.13583763, 0.1365017 , 0.13773385, 0.13632056]),
 'ideal_recall@10': array([0.91868049, 0.91868049, 0.91868049, 0.91868049, 0.91868049]),
 'nRecall@10': array([0.14527995, 0.14416427, 0.14498261, 0.14617821, 0.14483932]),
 'HR@20': array([0.5888836 , 0.586835  , 0.58777195, 0.58970939, 0.58759727]),
 'nDCG@20': array([0.13672262, 0.13626177, 0.13689224, 0.13710204, 0.1362932 ]),
 'recall@20':

A second improvement we explored was adding explicit feedback, in the form of game reviews. Not every user has reviewed every game they own, in fact only a tiny fraction of games is actually been reviewed. We append this explicit feedback from the user with sentiment rating, this sentiment rating is the average review score for a game. 

We explore two different sentiment metrics; mixed and rating

In mixed we use a combination of the average rating and the amount of users that own the game, this will introduce a bias to more popular games. 

In [8]:
evaluations = list()
weighting_scheme = {"playtime": False, "sentiment": "mixed", "reviews": True}
for split in range(am_splits):
    rec = ImprovedRecommender(items_path, train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet",
                              val_path=f"{data_path}{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf="smooth", normalize=True, weighting_scheme=weighting_scheme)
    rec.generate_recommendations(amount=am_recommended, silence=True)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
    rec.qualitative_evaluation(export_path=qual_output_path + f"ImprovedRec_sentiment_{split}.csv")    

evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "ImprovedRec_sentiment.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.33037955, 0.32887089, 0.32979196, 0.3306654 , 0.32686994]),
 'nDCG@5': array([0.10480854, 0.10401526, 0.10480852, 0.10467586, 0.10361621]),
 'recall@5': array([0.08723963, 0.08617528, 0.08687112, 0.08749612, 0.0855554 ]),
 'ideal_recall@5': array([0.78360788, 0.78360788, 0.78360788, 0.78360788, 0.78360788]),
 'nRecall@5': array([0.11397147, 0.11268858, 0.11345694, 0.11403155, 0.11191679]),
 'HR@10': array([0.45998094, 0.45875814, 0.46109258, 0.46171193, 0.46002859]),
 'nDCG@10': array([0.11648883, 0.11592154, 0.11688461, 0.11659989, 0.11606862]),
 'recall@10': array([0.1373888 , 0.13695778, 0.13772778, 0.13830169, 0.13712414]),
 'ideal_recall@10': array([0.91868049, 0.91868049, 0.91868049, 0.91868049, 0.91868049]),
 'nRecall@10': array([0.1464472 , 0.14587516, 0.14677326, 0.14725313, 0.14617134]),
 'HR@20': array([0.59321899, 0.59252025, 0.59277434, 0.59491822, 0.59479117]),
 'nDCG@20': array([0.13750168, 0.1369138 , 0.13779676, 0.1377271 , 0.13704559]),
 'recall@20':

For the rating only, we only use the average rating.

In [7]:
evaluations = list()
weighting_scheme = {"playtime": False, "sentiment": "rating", "reviews": True}
for split in range(am_splits):
    rec = ImprovedRecommender(items_path, train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet",
                              val_path=f"{data_path}{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf="smooth", normalize=True, weighting_scheme=weighting_scheme)
    rec.generate_recommendations(amount=am_recommended, silence=True)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
    rec.qualitative_evaluation(export_path=qual_output_path + f"ImprovedRec_sentiment_onlyrating_{split}.csv")    

evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "ImprovedRec_sentiment_onlyrating.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.32629824, 0.32628236, 0.32841035, 0.32710815, 0.32671113]),
 'nDCG@5': array([0.10514324, 0.10485217, 0.10512678, 0.10489374, 0.1045257 ]),
 'recall@5': array([0.08771734, 0.08717553, 0.08727922, 0.0876383 , 0.08656563]),
 'ideal_recall@5': array([0.78360788, 0.78360788, 0.78360788, 0.78360788, 0.78360788]),
 'nRecall@5': array([0.11364618, 0.11301228, 0.11319941, 0.1133958 , 0.11241438]),
 'HR@10': array([0.45704304, 0.45431158, 0.45820232, 0.46052088, 0.45861521]),
 'nDCG@10': array([0.11705043, 0.1163742 , 0.11718768, 0.11727909, 0.11681946]),
 'recall@10': array([0.1378476 , 0.13678147, 0.13802937, 0.13912031, 0.13783279]),
 'ideal_recall@10': array([0.91868049, 0.91868049, 0.91868049, 0.91868049, 0.91868049]),
 'nRecall@10': array([0.14664155, 0.14535823, 0.14674857, 0.14777342, 0.14657774]),
 'HR@20': array([0.59226616, 0.59101159, 0.59175798, 0.59391774, 0.59366365]),
 'nDCG@20': array([0.1381349 , 0.13765877, 0.13839269, 0.13854207, 0.13794225]),
 'recall@20':

Lastly we can include some implicit feedback from the user, this comes in the form of playtime. We use a logarithmic base 10 function to remove extremely large outliers, not doing this would make the recommender focus almost solely on that one game. By using logarithmic time, we can ensure that more games are being considered.

In [9]:
evaluations = list()
weighting_scheme = {"playtime": True, "sentiment": "mixed", "reviews": True}
for split in range(am_splits):
    rec = ImprovedRecommender(items_path, train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet",
                              val_path=f"{data_path}{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf="smooth", normalize=True, weighting_scheme=weighting_scheme)
    rec.generate_recommendations(amount=am_recommended, silence=True)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
    rec.qualitative_evaluation(export_path=qual_output_path + f"ImprovedRec_allweighting_{split}.csv")    

evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "ImprovedRec_allweighting.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.3109576 , 0.31000476, 0.31189455, 0.31116405, 0.30822614]),
 'nDCG@5': array([0.09575981, 0.09525407, 0.09556597, 0.09589899, 0.09462097]),
 'recall@5': array([0.07854396, 0.07749354, 0.07843343, 0.07877838, 0.07740133]),
 'ideal_recall@5': array([0.78360788, 0.78360788, 0.78360788, 0.78360788, 0.78360788]),
 'nRecall@5': array([0.10452702, 0.10338071, 0.10441321, 0.10456752, 0.10298899]),
 'HR@10': array([0.44024138, 0.43971733, 0.44165476, 0.43966968, 0.43952676]),
 'nDCG@10': array([0.10628156, 0.10582232, 0.10653833, 0.10645997, 0.10537771]),
 'recall@10': array([0.12474875, 0.12408138, 0.1258174 , 0.1251392 , 0.12397047]),
 'ideal_recall@10': array([0.91868049, 0.91868049, 0.91868049, 0.91868049, 0.91868049]),
 'nRecall@10': array([0.13398098, 0.13322246, 0.13498832, 0.1343103 , 0.13308374]),
 'HR@20': array([0.57447991, 0.57486104, 0.57284421, 0.57384469, 0.57484516]),
 'nDCG@20': array([0.12630515, 0.12587718, 0.12612137, 0.12650131, 0.1255619 ]),
 'recall@20':

In [8]:
evaluations = list()
weighting_scheme = {"playtime": True, "sentiment": "rating", "reviews": True}
for split in range(am_splits):
    rec = ImprovedRecommender(items_path, train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet",
                              val_path=f"{data_path}{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf="smooth", normalize=True, weighting_scheme=weighting_scheme)
    rec.generate_recommendations(amount=am_recommended, silence=True)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
    rec.qualitative_evaluation(export_path=qual_output_path + f"ImprovedRec_allweighting_onlyrating_{split}.csv")    

evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "ImprovedRec_allweighting_onlyrating.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.31041766, 0.31108464, 0.31125933, 0.31170399, 0.30767032]),
 'nDCG@5': array([0.09669435, 0.09645506, 0.09634136, 0.09693577, 0.09567491]),
 'recall@5': array([0.07921462, 0.07817158, 0.07875327, 0.08002048, 0.07783031]),
 'ideal_recall@5': array([0.78360788, 0.78360788, 0.78360788, 0.78360788, 0.78360788]),
 'nRecall@5': array([0.10474326, 0.10370123, 0.10428035, 0.10523794, 0.10299878]),
 'HR@10': array([0.44052724, 0.43955852, 0.44040019, 0.43806575, 0.4390821 ]),
 'nDCG@10': array([0.10712985, 0.10698062, 0.1071679 , 0.10727255, 0.1066283 ]),
 'recall@10': array([0.12570126, 0.12515823, 0.1259966 , 0.12599632, 0.12491978]),
 'ideal_recall@10': array([0.91868049, 0.91868049, 0.91868049, 0.91868049, 0.91868049]),
 'nRecall@10': array([0.1346102 , 0.13407104, 0.1349565 , 0.13489083, 0.13383939]),
 'HR@20': array([0.57459108, 0.5746546 , 0.57532158, 0.57298714, 0.57438463]),
 'nDCG@20': array([0.12722543, 0.1267316 , 0.12717404, 0.12735643, 0.12664675]),
 'recall@20':

# New data (v3)
We have observed that the previous dataset there occurs a heavy bias towards popular items. To combat this bias the top 1% of games has been removed.

In [10]:
data_basepath = "./data/v3/"
quant_output_path = "./evaluation/v3/quantitative/"
qual_output_path = "./evaluation/v3/qualitative/"
items_path = data_basepath + "games.pkl.gz"
reviews_path = data_basepath + "reviews.parquet"
data_path = data_basepath + "interactions_splits_"

## Popularity based recommender
Since top 1% has been removed, the popularity based recommender suffers a major performance hit.

In [12]:
evaluations = list()

for split in range(am_splits):
    rec = PopBasedRecommender(train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet", val_path=f"{data_path}{split}_val.parquet")
    rec.generate_recommendations(amount=am_recommended)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
        
evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "PopBased.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.11299133, 0.11686658, 0.11361875, 0.11391401, 0.11502122]),
 'nDCG@5': array([0.03173433, 0.03216759, 0.03139614, 0.03119367, 0.0317759 ]),
 'recall@5': array([0.02773729, 0.02886359, 0.02813285, 0.02721269, 0.03058411]),
 'ideal_recall@5': array([0.85553514, 0.85553514, 0.85553514, 0.85553514, 0.85553514]),
 'nRecall@5': array([0.03699053, 0.03813096, 0.03715476, 0.03650428, 0.03928431]),
 'HR@10': array([0.20806422, 0.21031556, 0.20854401, 0.20762133, 0.20965123]),
 'nDCG@10': array([0.0420735 , 0.04240384, 0.0419864 , 0.04137775, 0.04175161]),
 'recall@10': array([0.05927748, 0.05932461, 0.05935623, 0.05803128, 0.0590106 ]),
 'ideal_recall@10': array([0.94504488, 0.94504488, 0.94504488, 0.94504488, 0.94504488]),
 'nRecall@10': array([0.06391802, 0.06411943, 0.06413111, 0.06264198, 0.06379256]),
 'HR@20': array([0.3411146 , 0.33758996, 0.3378114 , 0.33670419, 0.33760841]),
 'nDCG@20': array([0.05853174, 0.05796137, 0.05752154, 0.05728064, 0.05768403]),
 'recall@20':

## Base recommender

In [14]:
evaluations = list()
use_data = ["genres", "tags"]
for split in range(am_splits):
    rec = ImprovedRecommender(items_path, train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet",
                              val_path=f"{data_path}{split}_val.parquet", reviews_path=reviews_path, tfidf="smooth", normalize=True, columns=use_data, weighting_scheme=None)
    rec.generate_recommendations(amount=am_recommended, silence=True)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
    rec.qualitative_evaluation(export_path=qual_output_path + f"BaseRecommender_{split}.csv")

evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "BaseRecommender.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.11891493, 0.11937627, 0.12114781, 0.11987452, 0.1215907 ]),
 'nDCG@5': array([0.04007563, 0.04036597, 0.04089923, 0.04068101, 0.04147375]),
 'recall@5': array([0.04094297, 0.04122622, 0.04167266, 0.04109895, 0.04162619]),
 'ideal_recall@5': array([0.85553514, 0.85553514, 0.85553514, 0.85553514, 0.85553514]),
 'nRecall@5': array([0.04713047, 0.04729378, 0.04786861, 0.04727594, 0.04798487]),
 'HR@10': array([0.18580919, 0.18601218, 0.18638125, 0.18669496, 0.1881528 ]),
 'nDCG@10': array([0.04715843, 0.04749399, 0.04797741, 0.04792445, 0.04860337]),
 'recall@10': array([0.06296349, 0.06331074, 0.06401075, 0.06365049, 0.06406515]),
 'ideal_recall@10': array([0.94504488, 0.94504488, 0.94504488, 0.94504488, 0.94504488]),
 'nRecall@10': array([0.06541333, 0.0657144 , 0.06640887, 0.06606534, 0.06649996]),
 'HR@20': array([0.27554899, 0.27453405, 0.27508765, 0.27486621, 0.27853848]),
 'nDCG@20': array([0.05774   , 0.0580072 , 0.05844182, 0.05844861, 0.0594297 ]),
 'recall@20':

## Improved recommender

Increased item representation on v3 dataset

In [15]:
evaluations = list()
for split in range(am_splits):
    rec = ImprovedRecommender(items_path, train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet",
                              val_path=f"{data_path}{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf="smooth", normalize=True)
    rec.generate_recommendations(amount=am_recommended, silence=True)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
    rec.qualitative_evaluation(export_path=qual_output_path + f"ImprovedRec_none_{split}.csv")    

evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "ImprovedRec_none.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.18407455, 0.18568001, 0.18564311, 0.18453589, 0.18617826]),
 'nDCG@5': array([0.06909971, 0.06978816, 0.07075082, 0.06970324, 0.06991053]),
 'recall@5': array([0.06958126, 0.06961221, 0.07105591, 0.06970356, 0.07006132]),
 'ideal_recall@5': array([0.85553514, 0.85553514, 0.85553514, 0.85553514, 0.85553514]),
 'nRecall@5': array([0.07830565, 0.07846497, 0.07982223, 0.07858892, 0.07893092]),
 'HR@10': array([0.26006643, 0.26019561, 0.26001107, 0.25663407, 0.26148736]),
 'nDCG@10': array([0.07735403, 0.07774755, 0.07858462, 0.07731974, 0.07798247]),
 'recall@10': array([0.09711465, 0.09667349, 0.09737175, 0.09574575, 0.09753496]),
 'ideal_recall@10': array([0.94504488, 0.94504488, 0.94504488, 0.94504488, 0.94504488]),
 'nRecall@10': array([0.10026047, 0.09975722, 0.10042289, 0.09888567, 0.10063477]),
 'HR@20': array([0.35401366, 0.35482561, 0.35510242, 0.34936335, 0.35890386]),
 'nDCG@20': array([0.08945398, 0.08996689, 0.09082602, 0.08931702, 0.09044149]),
 'recall@20':

Adding explicit feedback from reviews, appended with sentiment rating. 

In [16]:
evaluations = list()
weighting_scheme = {"playtime": False, "sentiment": "rating", "reviews": True}
for split in range(am_splits):
    rec = ImprovedRecommender(items_path, train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet",
                              val_path=f"{data_path}{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf="smooth", normalize=True, weighting_scheme=weighting_scheme)
    rec.generate_recommendations(amount=am_recommended, silence=True)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
    rec.qualitative_evaluation(export_path=qual_output_path + f"ImprovedRec_sentiment_{split}.csv")    

evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "ImprovedRec_sentiment.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.18276435, 0.18601218, 0.18610445, 0.18390847, 0.18798671]),
 'nDCG@5': array([0.06898706, 0.06992499, 0.07061812, 0.06964751, 0.07061694]),
 'recall@5': array([0.06881763, 0.06961669, 0.07074795, 0.06916052, 0.070382  ]),
 'ideal_recall@5': array([0.85553514, 0.85553514, 0.85553514, 0.85553514, 0.85553514]),
 'nRecall@5': array([0.07760472, 0.0786006 , 0.07959833, 0.07805468, 0.07943009]),
 'HR@10': array([0.25825798, 0.25875623, 0.25862705, 0.25729839, 0.26076767]),
 'nDCG@10': array([0.07697784, 0.07732634, 0.07811178, 0.07734465, 0.07809618]),
 'recall@10': array([0.09574987, 0.09548798, 0.09655263, 0.09544171, 0.09682705]),
 'ideal_recall@10': array([0.94504488, 0.94504488, 0.94504488, 0.94504488, 0.94504488]),
 'nRecall@10': array([0.09885495, 0.09853197, 0.0995738 , 0.0985698 , 0.09993603]),
 'HR@20': array([0.3529618 , 0.35288799, 0.35242665, 0.34820078, 0.35619118]),
 'nDCG@20': array([0.08925301, 0.08946571, 0.09024228, 0.08896525, 0.0902733 ]),
 'recall@20':

Adding play time as implicit feedback

In [17]:
evaluations = list()
weighting_scheme = {"playtime": True, "sentiment": "rating", "reviews": True}
for split in range(am_splits):
    rec = ImprovedRecommender(items_path, train_path=f"{data_path}{split}_train.parquet", test_path=f"{data_path}{split}_test.parquet",
                              val_path=f"{data_path}{split}_val.parquet", reviews_path=reviews_path, sparse=True, tfidf="smooth", normalize=True, weighting_scheme=weighting_scheme)
    rec.generate_recommendations(amount=am_recommended, silence=True)
    evaluation = dict()
    for k in k_values:
        evaluation.update(rec.evaluate(val=False, k=k))
    evaluations.append(evaluation)
    rec.qualitative_evaluation(export_path=qual_output_path + f"ImprovedRec_allweighting_{split}.csv")    

evaluations = invert_dict(evaluations)
save_pickle(quant_output_path + "ImprovedRec_allweighting.pickle.gz", evaluations)
evaluations

{'HR@5': array([0.15017531, 0.14999077, 0.1524451 , 0.14976933, 0.1519284 ]),
 'nDCG@5': array([0.05384453, 0.05277788, 0.05416939, 0.05308758, 0.05376281]),
 'recall@5': array([0.05307538, 0.05189343, 0.05386586, 0.05232132, 0.05337799]),
 'ideal_recall@5': array([0.85553514, 0.85553514, 0.85553514, 0.85553514, 0.85553514]),
 'nRecall@5': array([0.06131636, 0.06011872, 0.0619896 , 0.06048287, 0.06170726]),
 'HR@10': array([0.21902565, 0.21935782, 0.22101864, 0.21789998, 0.22216276]),
 'nDCG@10': array([0.06052951, 0.05987267, 0.06098913, 0.05964197, 0.06084916]),
 'recall@10': array([0.07631429, 0.07625222, 0.07689053, 0.07476273, 0.07742829]),
 'ideal_recall@10': array([0.94504488, 0.94504488, 0.94504488, 0.94504488, 0.94504488]),
 'nRecall@10': array([0.07948651, 0.0794094 , 0.08007647, 0.07796026, 0.08062834]),
 'HR@20': array([0.31088762, 0.31096143, 0.31332349, 0.30789814, 0.31251153]),
 'nDCG@20': array([0.07137168, 0.07075025, 0.0720439 , 0.07031909, 0.07151046]),
 'recall@20':